In [1]:
import torch
from torch import nn, einsum
import lovely_tensors as lt
lt.monkey_patch()

In [2]:
inp = torch.randn(1, 80, 512)

In [3]:
k_u = [10, 20, 30]

In [6]:
prev_ch = 80
new_ch = 512
stride = 128
conv_init = nn.Conv1d(prev_ch, new_ch, kernel_size=3, stride=1, padding=1)

In [12]:
prev_ch = new_ch
new_ch = int(new_ch/2)
stride = int(stride/2)
first = nn.ConvTranspose1d(prev_ch, new_ch, kernel_size=3, stride=stride)

In [15]:
nn.ConvTranspose1d(in_channels=3, out_channels=1, kernel_size=[7,1])

ConvTranspose1d(3, 1, kernel_size=(7, 1), stride=(1,))

In [17]:
test = nn.Conv1d(3, 4, 3)
a = torch.randn(1, 3,  10)
test(a)

tensor[1, 4, 8] n=32 x∈[-1.185, 1.046] μ=-0.030 σ=0.531 grad ConvolutionBackward0

In [2]:
class ResBlk(nn.Module):
    def __init__(self, kernel, dilations, channel):
        super().__init__()
        layers = []
        for l in range(len(dilations)):
            layers.append(nn.LeakyReLU())
            layers.append(nn.Conv1d(in_channels=channel, out_channels=channel,
                                    kernel_size=kernel, dilation=dilations[l]))


        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [4]:
class MRF(nn.Module):
    def __init__(self, k_r, D_r, channel):
        super().__init__()
        self.k_r = k_r
        self.D_r = D_r
        layers = []
        for n in range(len(D_r)):
            layers.append(ResBlk(k_r[n], D_r[n], channel))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        h = x
        for layer in self.layers:
            h=layer(h)+h

        return h

In [32]:
t = torch.randn(1,3,32)
l1 = nn.Conv1d(3, 10, kernel_size=3,stride=1,padding=1, dilation=2)
l2 = nn.Conv1d(10, 20, kernel_size=3, stride=1, padding=1, dilation=4)

skip = nn.Conv1d(3, 20, kernel_size=1, )

In [35]:
l2(l1(t))

tensor[1, 20, 24] n=480 x∈[-1.153, 1.173] μ=-0.014 σ=0.377 grad ConvolutionBackward0

In [41]:
def get_padding(kernel, dilation):
    return int((kernel*dilation - dilation)/2)

In [42]:
# padding을 적절히 조절해서 shape을 맞추도록 함
l1 = nn.Conv1d(3, 3, kernel_size=3, stride=1, dilation=1, padding=get_padding(3, 1))
l2 = nn.Conv1d(3, 3, kernel_size=3, stride=1, dilation=1, padding=get_padding(3, 1))

In [43]:
t = torch.randn(1,3,32)
l2(l1(t))

tensor[1, 3, 32] n=96 x∈[-1.083, 0.763] μ=-0.390 σ=0.376 grad ConvolutionBackward0

In [ ]:
class ResBlk(nn.Module):
    def __init__(self, channels, kernel_size, dilation=(1,3,5)):
        super().__init__()
        self.conv1 = nn.ModuleList(nn.Conv1d(),
                                   nn.Conv1d(),
                                   nn.Conv1d())
        self.conv2 = nn.ModuleList(nn.Conv1d(),
                                   nn.Conv1d(),
                                   nn.Conv1d())

    def forward(self, x):
        for c1, c2 in zip(self.conv1, self.conv2):
            

In [8]:
k_u = [16,16,4,4]
k_r = [3,7,11]
h_u = 512
layers = []
D_r = [[1, 1], [3, 1], [5, 1]]
channel = 32
test = MRF(k_r, D_r, channel)

In [10]:
test

MRF(
  (layers): Sequential(
    (0): ResBlk(
      (layers): Sequential(
        (0): LeakyReLU(negative_slope=0.01)
        (1): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (2): LeakyReLU(negative_slope=0.01)
        (3): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
    )
    (1): ResBlk(
      (layers): Sequential(
        (0): LeakyReLU(negative_slope=0.01)
        (1): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(3,))
        (2): LeakyReLU(negative_slope=0.01)
        (3): Conv1d(32, 32, kernel_size=(7,), stride=(1,))
      )
    )
    (2): ResBlk(
      (layers): Sequential(
        (0): LeakyReLU(negative_slope=0.01)
        (1): Conv1d(32, 32, kernel_size=(11,), stride=(1,), dilation=(5,))
        (2): LeakyReLU(negative_slope=0.01)
        (3): Conv1d(32, 32, kernel_size=(11,), stride=(1,))
      )
    )
  )
)

In [9]:
test(torch.randn(1, 32, 100))

RuntimeError: The size of tensor a (96) must match the size of tensor b (100) at non-singleton dimension 2

In [6]:
k_u = [16,16,4,4]
k_r = [3,7,11]
h_u = 512
layers = []
D_r = [[1, 1], [3, 1], [5, 1]]

layers.append(nn.Conv1d(in_channels=1, out_channels=h_u, kernel_size=[7,1], dilation=1))
prev_ch = h_u
for l in range(len(k_u)):
    new_ch = prev_ch //2
    layers.append(nn.LeakyReLU())
    stride = k_u[l]/2
    layers.append(nn.ConvTranspose1d(prev_ch, new_ch, kernel_size=3, stride=stride))
    layers.append(MRF(k_r, D_r, new_ch))
    prev_ch = new_ch 
layers.append(nn.LeakyReLU())
layers.append(nn.Conv1d(in_channels=prev_ch, out_channels=1, kernel_size=[7,1]))
layers.append(nn.Tanh())

In [7]:
nn.Sequential(*layers)

Sequential(
  (0): Conv1d(1, 512, kernel_size=(7, 1), stride=(1,))
  (1): LeakyReLU(negative_slope=0.01)
  (2): ConvTranspose1d(512, 256, kernel_size=(3,), stride=(8.0,))
  (3): MRF(
    (layers): Sequential(
      (0): ResBlk(
        (layers): Sequential(
          (0): LeakyReLU(negative_slope=0.01)
          (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
          (2): LeakyReLU(negative_slope=0.01)
          (3): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
        )
      )
      (1): ResBlk(
        (layers): Sequential(
          (0): LeakyReLU(negative_slope=0.01)
          (1): Conv1d(256, 256, kernel_size=(7,), stride=(1,), dilation=(3,))
          (2): LeakyReLU(negative_slope=0.01)
          (3): Conv1d(256, 256, kernel_size=(7,), stride=(1,))
        )
      )
      (2): ResBlk(
        (layers): Sequential(
          (0): LeakyReLU(negative_slope=0.01)
          (1): Conv1d(256, 256, kernel_size=(11,), stride=(1,), dilation=(5,))
          (2): LeakyReLU(negativ

In [ ]:
class Generator(nn.Module):
    def __init__(self, )

In [ ]:
from torch.nn.utils import weight_norm, 

In [2]:
import torch

def index(feat, uv):
    '''
    extract image features at floating coordinates with bilinear interpolation
    args:
        feat: [B, C, H, W] image features
        uv: [B, 2, N] normalized image coordinates ranged in [-1, 1]
    return:
        [B, C, N] sampled pixel values
    '''
    uv = uv.transpose(1, 2)
    uv = uv.unsqueeze(2)
    samples = torch.nn.functional.grid_sample(feat, uv, align_corners=True)
    return samples[:, :, :, 0]

In [7]:
import torch
import torch.nn.functional as F

# Define an input tensor with size (1, 1, 3, 3)
input_tensor = torch.arange(25).reshape((1,1,5,5))
input_tensor.v

tensor[1, 1, 5, 5] i64 n=25 x∈[0, 24] μ=12.000 σ=7.360
tensor([[[[ 0,  1,  2,  3,  4],
          [ 5,  6,  7,  8,  9],
          [10, 11, 12, 13, 14],
          [15, 16, 17, 18, 19],
          [20, 21, 22, 23, 24]]]])

In [ ]:

# Define a transformation grid with size (1, 2, 3)
grid = torch.tensor([[[0.5, 0.5, 0], [0.5, 0.5, 0]]])

# Apply the grid_sample function
output = F.grid_sample(input_tensor, grid)

# Print the output
print(output)